In [2]:
#%reset
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk
import numpy as np
import eli5

from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import cross_validate, RandomizedSearchCV, RepeatedStratifiedKFold
from sklearn.utils import resample
from sklearn.inspection import permutation_importance
from scipy.stats import loguniform
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import confusion_matrix
from eli5.sklearn import PermutationImportance
from sklearn.metrics import f1_score, matthews_corrcoef

hfont = {'fontname':'Helvetica'}

## read data 
CIP_data = pd.read_csv("CIP_data_encode_prev.csv")
CIP_data_no_drop = pd.read_csv("CIP_data_encode_prev_not_dropped.csv")
print(CIP_data_no_drop.columns)

Index(['Unnamed: 0.4', 'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1',
       'Unnamed: 0', 'CLINIC', 'YEAR', 'GENDERSP', 'Susceptible', 'MSM',
       'MSMW', 'MSW', 'Oth/Unk/Missing', 'REGION', 'Midwest', 'Northeast',
       'Southeast', 'Southwest', 'West', 'PREV_REGION', 'PREV_CLINIC',
       'DELTA_REGION', 'DELTA_CLINIC'],
      dtype='object')


In [3]:

#nn_data 
best_features_by_year_nn = {2005: ['PREV_REGION', 'West', 'Midwest', 'DELTA_REGION', 'DELTA_CLINIC', 'Southeast', 'Southwest', 'MSW'], 2006: ['PREV_CLINIC', 'PREV_REGION', 'DELTA_CLINIC', 'MSM', 'DELTA_REGION', 'Midwest'], 2007: ['PREV_CLINIC', 'PREV_REGION', 'MSW', 'DELTA_REGION', 'Midwest', 'MSMW', 'Oth/Unk/Missing'], 2008: ['PREV_REGION', 'PREV_CLINIC', 'DELTA_CLINIC', 'MSW', 'West', 'DELTA_REGION', 'MSM', 'Midwest', 'Northeast', 'MSMW'], 2009: ['PREV_CLINIC', 'DELTA_REGION', 'MSW', 'DELTA_CLINIC'], 2010: ['PREV_CLINIC', 'MSW', 'MSM', 'DELTA_CLINIC', 'MSMW', 'PREV_REGION', 'West', 'Southeast', 'Southwest', 'Oth/Unk/Missing']}
best_hyperparameters_by_year_nn = {2005: {'solver': 'sgd', 'learning_rate': 'constant', 'hidden_layer_sizes': 12, 'alpha': 0.12618568830660204, 'activation': 'tanh'}, 2006: {'solver': 'sgd', 'learning_rate': 'constant', 'hidden_layer_sizes': 12, 'alpha': 0.12618568830660204, 'activation': 'tanh'}, 2007: {'solver': 'sgd', 'learning_rate': 'constant', 'hidden_layer_sizes': 12, 'alpha': 0.12618568830660204, 'activation': 'tanh'}, 2008: {'solver': 'sgd', 'learning_rate': 'constant', 'hidden_layer_sizes': 12, 'alpha': 0.12618568830660204, 'activation': 'tanh'}, 2009: {'solver': 'sgd', 'learning_rate': 'constant', 'hidden_layer_sizes': 12, 'alpha': 0.12618568830660204, 'activation': 'tanh'}, 2010: {'solver': 'sgd', 'learning_rate': 'constant', 'hidden_layer_sizes': 12, 'alpha': 0.12618568830660204, 'activation': 'tanh'}}
ROC_by_year_nn = {2005: 0.707710113960114, 2006: 0.7363098705025091, 2007: 0.6839585775693631, 2008: 0.6843168957154405, 2009: 0.6645239980510848, 2010: 0.6755020080321286}##lr_data 

#lr_data 
best_hyperparameters_by_year_lr = {2005: {'solver': 'liblinear', 'penalty': 'l1', 'C': 8.65}, 2006: {'solver': 'liblinear', 'penalty': 'l1', 'C': 35.730000000000004}, 2007: {'solver': 'liblinear', 'penalty': 'l1', 'C': 8.65}, 2008: {'solver': 'liblinear', 'penalty': 'l1', 'C': 54.85}, 2009: {'solver': 'liblinear', 'penalty': 'l2', 'C': 83.42}, 2010: {'solver': 'liblinear', 'penalty': 'l1', 'C': 8.65}}
best_features_by_year_lr = {2005: ['PREV_CLINIC', 'MSM', 'DELTA_CLINIC', 'PREV_REGION', 'MSW', 'Southwest', 'Northeast', 'Oth/Unk/Missing'], 2006: ['DELTA_REGION', 'PREV_CLINIC', 'DELTA_CLINIC', 'MSW', 'PREV_REGION', 'Oth/Unk/Missing', 'MSM', 'Southwest', 'Southeast'], 2007: ['MSM', 'PREV_CLINIC', 'MSW', 'Oth/Unk/Missing', 'MSMW'], 2008: ['PREV_CLINIC', 'DELTA_CLINIC', 'West', 'MSW', 'MSM', 'PREV_REGION', 'MSMW', 'Oth/Unk/Missing', 'Northeast'], 2009: ['PREV_CLINIC', 'Oth/Unk/Missing', 'DELTA_CLINIC', 'Northeast'], 2010: ['MSW', 'MSM', 'DELTA_CLINIC', 'PREV_CLINIC', 'Oth/Unk/Missing', 'West', 'Southwest', 'MSMW', 'DELTA_REGION', 'Southeast', 'Northeast']}
ROC_by_year_lr = {2005: 0.7317511805463615, 2006: 0.7423710317796873, 2007: 0.7048918256421187, 2008: 0.6971529715492738, 2009: 0.6166128405256475, 2010: 0.6792793175522966}
##rf_paper 
best_hyperparameters_by_year_rf = {2005: {'n_estimators': 163, 'min_samples_split': 5, 'min_samples_leaf': 14, 'max_depth': 81}, 2006: {'n_estimators': 163, 'min_samples_split': 5, 'min_samples_leaf': 14, 'max_depth': 81}, 2007: {'n_estimators': 163, 'min_samples_split': 5, 'min_samples_leaf': 14, 'max_depth': 81}, 2008: {'n_estimators': 163, 'min_samples_split': 5, 'min_samples_leaf': 14, 'max_depth': 81}, 2009: {'n_estimators': 163, 'min_samples_split': 5, 'min_samples_leaf': 14, 'max_depth': 81}, 2010: {'n_estimators': 163, 'min_samples_split': 5, 'min_samples_leaf': 14, 'max_depth': 81}}
best_features_by_year_rf = {2005: ['MSW', 'MSM', 'MSMW'], 2006: ['MSW', 'DELTA_CLINIC', 'MSM', 'MSMW', 'Oth/Unk/Missing'], 2007: ['MSW', 'PREV_CLINIC', 'MSM'], 2008: ['PREV_CLINIC', 'MSW', 'MSM', 'DELTA_CLINIC', 'Oth/Unk/Missing', 'Southeast'], 2009: ['MSW', 'MSM', 'PREV_CLINIC', 'DELTA_CLINIC', 'PREV_REGION', 'Midwest'], 2010: ['DELTA_CLINIC', 'PREV_CLINIC', 'MSW', 'PREV_REGION', 'West', 'DELTA_REGION', 'Midwest', 'Southeast', 'MSMW']}
ROC_by_year_rf = {2005: 0.7418091168091168, 2006: 0.6955865853135346, 2007: 0.6980265728800312, 2008: 0.7012934518997574, 2009: 0.6612742833389195, 2010: 0.6738955823293172}

In [7]:
########### Try functionalised 


def effective_unnecessary_threshold(threshold_seq, y_predict_proba, y_test, cipro_R_prevalence):

    get_effective_threshold = []
    incorrectly_get_X_threshold = [] #no bootstrapping, no 95% CI 
    sensitivity_threshold = []
    specificity_threshold = []
    for threshold in threshold_seq:

        y_predict_test = np.where(y_predict_proba[:, 1] > threshold, 1, 0)

        tn_test , fp_test , fn_test , tp_test  = confusion_matrix(y_true=y_test, y_pred=y_predict_test).ravel()

        sensitivity_test  = tp_test  / (tp_test   + fn_test )
        specificity_test   = tn_test / (tn_test + fp_test )

        sensitivity_threshold.append(sensitivity_test*100)
        specificity_threshold.append(specificity_test*100)
        get_effective_threshold.append(sensitivity_test * cipro_R_prevalence*100 + (100 -  cipro_R_prevalence*100)) #q_p
        incorrectly_get_X_threshold.append((100 - cipro_R_prevalence*100) * (1 - specificity_test)) #c_p"
    return(sensitivity_threshold, specificity_threshold, get_effective_threshold, incorrectly_get_X_threshold)

def get_best_hyperparameters(model, cv, space, X_train, y_train):
        search = RandomizedSearchCV(model, space, scoring='roc_auc', n_iter=1,  n_jobs=-1, cv=cv, random_state=1)
        result = search.fit(X_train, y_train)
        return(result.best_params_)

def get_best_features(feature_names, model_fit, X_test, y_test):
    PI = permutation_importance(model_fit, X_test, y_test, n_repeats = 10, random_state = 42)
    important_features = []
    for q in PI.importances_mean.argsort()[::-1]:
        if PI.importances_mean[q] - 2 * PI.importances_std[q] > 0:
          important_features.append(feature_names[q]) #works cos they are in same order as the x columns
    return(important_features)


def get_feature_effects(feature_names, model_fit, X_test, y_test):
    PI = permutation_importance(model_fit, X_test, y_test, n_repeats = 10, random_state = 42)
    
    return(PI.importances_mean)

oversample = RandomOverSampler(sampling_strategy = 0.5, random_state=42)

def get_test_train_data_overfit(CIP_data_no_drop, year, feature_names, oversample_size):
    years_train = np.array(range(year - 5, year))

    # first do for all clinics 
    train_data = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin(years_train)]
    X_train = train_data[feature_names] #need to consider all columns BEFORE feature engineering
    y_train = 1 - train_data['Susceptible']
    X_train, y_train = oversample.fit_resample(X_train,y_train)
    #test
    test_data = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin([year])]
    X_test = test_data[feature_names]
    y_test = 1 - test_data['Susceptible']
    cipro_R = y_test.sum()/len(y_test) 
    X_test, y_test = oversample.fit_resample(X_test,y_test)
    return(test_data, train_data, X_train, y_train, X_test, y_test, cipro_R)


def get_test_train_data(CIP_data_no_drop, year, feature_names):
    years_train = np.array(range(year - 5, year))

    CIP_data_training_years = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin(years_train)]
    CIP_data_testing_years = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin([year])]
    # first do for all clinics 
    train_data = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin(years_train)]
    X_train = train_data[feature_names] #need to consider all columns BEFORE feature engineering
    y_train = 1 - train_data['Susceptible']
    #test
    test_data = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin([year])]
    X_test = test_data[feature_names]
    y_test = 1 - test_data['Susceptible']
    return(test_data, train_data, X_train, y_train, X_test, y_test)

In [10]:
########### Now do proportion receiving effective and unnecessary treatment for years surrounding 2007 decision 
### now do a loop over different years to get the proportion receiving effective and unnecessary treatment 
### has LOOCV 

## NEEDED FOR FEATURE ENGINEERING 
feature_names = ['MSM','MSMW', 'MSW', 'Oth/Unk/Missing','Northeast', 'Southeast', 'Southwest', 'West', 'Midwest','PREV_REGION', 'PREV_CLINIC', 'DELTA_REGION', 'DELTA_CLINIC']


#### Loop set up 
test_years = [2005, 2006, 2007, 2008, 2009, 2010]


### effect features by year
best_effect_by_year_lr = {}
best_effect_by_year_nn = {}
best_effect_by_year_rf = {}

for year in test_years: 
    years_train = np.array(range(year - 5, year))

    CIP_data_training_years = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin(years_train)]
    CIP_data_testing_years = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin([year])]
    
    ## LOGISTIC REGRESSION - no overfit 
    test_data, train_data, X_train, y_train, X_test, y_test =  get_test_train_data(CIP_data_no_drop = CIP_data_no_drop, year = year, feature_names = feature_names)
    cipro_R_prev = y_test.sum()/len(y_test) #get prevalence for all clinics for "general" model for year 
    model_lr = LogisticRegression(class_weight = 'balanced', max_iter=4000, solver = best_hyperparameters_by_year_lr[year]['solver'], C = best_hyperparameters_by_year_lr[year]['C'], penalty = best_hyperparameters_by_year_lr[year]['penalty'])

    ## fit model w/hyperparameters 
    model_fit_lr = model_lr.fit(X_train, y_train)
    ## now also need to do feature engineering
    features_effects_lr = get_feature_effects(feature_names, model_fit_lr, X_test, y_test)
    best_effect_by_year_lr.__setitem__(year, features_effects_lr) 

    ## NEURAL NETWORK 
    test_data, train_data, X_train, y_train, X_test, y_test, cipro_R_prev =  get_test_train_data_overfit(CIP_data_no_drop = CIP_data_no_drop, year = year, feature_names = best_features_by_year_nn[year], oversample_size = 0.5)

    model_nn = MLPClassifier(solver = 'lbfgs', activation = 'tanh', max_iter = 3000 ,hidden_layer_sizes= best_hyperparameters_by_year_nn[year]['hidden_layer_sizes'], alpha =  best_hyperparameters_by_year_nn[year]['alpha'], random_state=10, learning_rate = 'adaptive' )
        ## fit model w/hyperparameters 
    model_fit_nn = model_nn.fit(X_train, y_train)
    ## now also need to do feature engineering
    features_effects_nn = get_feature_effects(feature_names, model_fit_nn, X_test, y_test)
    best_effect_by_year_nn.__setitem__(year, features_effects_nn) 

    ## RANDOM FOREST 

    model_rf = RandomForestClassifier(n_estimators = best_hyperparameters_by_year_rf[year]['n_estimators'], min_samples_split = best_hyperparameters_by_year_rf[year]['min_samples_split'], min_samples_leaf=best_hyperparameters_by_year_rf[year]['min_samples_leaf'], max_features = 'sqrt', max_depth = best_hyperparameters_by_year_rf[year]['max_depth'], random_state = 10)
    model_fit_rf = model_rf.fit(X_train, y_train)
    ## now also need to do feature engineering
    features_effects_rf = get_feature_effects(feature_names, model_fit_rf, X_test, y_test)
    best_effect_by_year_rf.__setitem__(year, features_effects_rf) 



    

In [ ]:
### Make a dataframe of results 

imporances_all_models = pd.Dataframe(0, rows = feature_names, columns=np.arange(len(feature_names)*3))


In [9]:
print(best_effect_by_year_rf)
print(best_effect_by_year_nn)
print(best_effect_by_year_lr)

{2005: array([ 0.00506697,  0.        ,  0.03299984,  0.00456673,  0.        ,
       -0.00932709,  0.        , -0.00921414, -0.010731  ,  0.0182669 ,
        0.04471518, -0.0211554 ,  0.01929966]), 2006: array([ 1.50788436e-02,  5.58475690e-04,  1.63436268e-02,  2.69382392e-03,
        4.23455979e-02,  0.00000000e+00,  6.35676741e-03, -3.12089356e-04,
       -1.64257556e-05,  2.36859396e-02,  1.11990802e-01,  1.14520368e-01,
        7.56570302e-02]), 2007: array([ 0.02622733,  0.00244633,  0.01860543,  0.00485938,  0.00024963,
        0.        ,  0.00029955, -0.01750707, -0.01399567, -0.0236645 ,
        0.02822433, -0.0314029 , -0.04479947]), 2008: array([ 0.02042635,  0.0019046 ,  0.02295999,  0.00108335,  0.00022715,
       -0.0007863 ,  0.00013979,  0.03706098, -0.0002621 ,  0.00503233,
        0.06978857, -0.0013105 ,  0.04868076]), 2009: array([-2.03058321e-02, -8.32147937e-03, -1.66251778e-02,  8.49928876e-03,
        2.84495021e-04, -1.08463727e-03,  3.55618777e-05, -5.849928